<a href="https://colab.research.google.com/github/Manish533005/nlp_chatbot/blob/main/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [90]:
import pickle
import numpy as np
from keras.models import load_model
import json
import random
from fuzzywuzzy import process
import spacy
model =load_model("chatbot_6.h5")
intents = json.loads(open('chatbot_dataset.json').read())
intents_1=json.loads(open("food_document.json").read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))
nlp=spacy.load("en_core_web_sm")
print(len(words))
print(len(classes))

45
14


In [91]:
!pip install fuzzywuzzy

In [92]:
print(intents_1)

{'intents': [{'tag': 'Non-Veg Pizza', 'items': ['chicken pepperoni', 'Pepper Barbecue', 'Dominator', 'chicken Sausage', 'pepper barbecue and onion', 'Indi chicken tikka', 'Moroccan Spice Pasta Pizza'], 'size': ['small', 'medium', 'large'], 'price': [300, 550, 830]}, {'tag': 'Veg Pizza', 'items': ['peppy paneer', 'Mexican green wave', 'Veggie paradise', 'veg extravaganza', 'deluxe veggie', 'paneer Makhani', 'creamy tomato pasta '], 'size': ['small', 'medium', 'large'], 'price': [250, 499, 799]}, {'tag': 'Beverages', 'items': ['pepsi', 'slice', '7up', 'mirinda', 'nimbooz'], 'size': [], 'price': [50]}, {'tag': 'Slides', 'items': ['garlic breadsticks', 'taco mexicana veg', 'Butterscotch mouse cake'], 'size': [], 'price': [99]}]}


In [93]:
print(words)

['-', '-PRON-', 'add', 'available', 'beverage', 'bye', 'cancel', 'cancle', 'change', 'chat', 'chicken', 'combo', 'confirm', 'cool', 'dominator', 'drink', 'edit', 'food', 'goodbye', 'hard', 'hello', 'hey', 'hi', 'hungry', 'item', 'large', 'later', 'medium', 'menu', 'nice', 'non', 'nonveg', 'order', 'pizza', 'remove', 'request', 'slide', 'small', 'till', 'time', 'today', 'variety', 'veg', 'view', 'want']


In [94]:
all_stopwords = nlp.Defaults.stop_words
sentence_2=[]
sent=[]

In [95]:
#tokenize each word
def fuzzy(sentence):
    str2match=sentence
    strOptions=words
    Ratios=process.extract(str2match,strOptions)
    highest=process.extractOne(str2match,strOptions)
    return highest[0]
def clean_up_sentence(sentence):
        sentence_2=[]
        sent=[]
        sentence=str(sentence)
        sentence_1=fuzzy(sentence)
        sentence_words= nlp(sentence_1)
        l=[]
        for i in sentence_words:
             i=str(i)
             l.append(i)
      
        sentence_2.extend(l)
        # lemmatizing the words
        sent=[nlp(word.lower())[0].lemma_ for word in sentence_2]
        sent=[ w  for w in sent if w not in all_stopwords]
        return sent
def bow(sentence ,words):

    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list    

In [96]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result


def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    tag = ints[0]['intent']
    if tag=="order_food":
       k=print_menu()
       res=res+" \n "+ k[0] +" \n "+ k[1] +" \n "+ k[2] +" \n "+k[3]
    elif tag=="Non-Veg Pizza":
       q=print_nonveg()
       res=res+" \n "+ q[0] +" \n "+ q[1] +" \n "+ q[2] +" \n "+q[3]+" \n "

    elif tag=="Veg Pizza":
        v=print_veg()
        res=res+" \n "+ v[0] +" \n "+ v[1] +" \n "+ v[2] +" \n "+v[3]+" \n "
    elif tag=="Beverages":
         B=print_bever()
         res=res+" \n "+ B[0] +" \n "+ B[1] +" \n "+ B[2] +" \n "+B[3]+" \n "
    elif tag=="Slides":
         s=print_slides()
         res=res+" \n "+ s[0] +" \n "+ s[1] +" \n "+ s[2] +" \n "

    return res

In [97]:
def print_nonveg():
    food=[]
    for intent in intents_1['intents']:
        if intent['tag']=="Non-Veg Pizza":
          for i in intent['items']:
            food.append(i)        
    return food       

In [98]:
#method to print the menu
def print_menu():
  l=[]
  for intent in intents_1['intents']:
      l.append(intent['tag'])
  return l

In [99]:
def print_veg():
    food=[]
    for intent in intents_1['intents']:
        if intent['tag']=="Veg Pizza":
            for i in intent['items']:
                food.append(i)        
    return food
print_veg()     

['peppy paneer',
 'Mexican green wave',
 'Veggie paradise',
 'veg extravaganza',
 'deluxe veggie',
 'paneer Makhani',
 'creamy tomato pasta ']

In [100]:
def print_bever():
    food=[]
    for intent in intents_1['intents']:
        if intent['tag']=="Beverages":
            for i in intent['items']:
                food.append(i)        
    return food
print_bever()    

['pepsi', 'slice', '7up', 'mirinda', 'nimbooz']

In [101]:
def print_slides():
    food=[]
    for intent in intents_1['intents']:
        if intent['tag']=="Slides":
            for i in intent['items']:
                food.append(i)        
    return food
print_slides()    

['garlic breadsticks', 'taco mexicana veg', 'Butterscotch mouse cake']

In [102]:
#method to print the non veg menu
def non_veg():
    for intent in intents_1['intents']:
      k=intent['tag']
      print(k)


In [117]:
k=chatbot_response("cancel order")
print(k)

your order cancelled successfully
